In [1]:
import Classifiers.SVMClassifier as svm
import Preprocessers.PCA as pca

import Kernels.LinearKernel as lker
import Kernels.GaussianKernel as gker
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from mpl_toolkits.mplot3d import Axes3D

filepath = 'challenge-dataset/'

KeyboardInterrupt: 

In [ ]:
def bound_normalisation(x):
    if x == 0:
        return -1
    else:
        return x

In [ ]:
#DATA LOADING
Xtr0_mat100 = pd.read_csv(filepath + 'Xtr0_mat100.csv', delimiter=' ', header = None)
Xte0_mat100 = pd.read_csv(filepath + 'Xte0_mat100.csv', delimiter=' ', header = None)

Ytr0 = pd.read_csv(filepath + 'Ytr0.csv', delimiter=',', sep='\n', header = 0)
Ytr0['Bound'] = Ytr0['Bound'].map(bound_normalisation)


In [ ]:
#SLICING BETWEEN TEST AND TRAINING
inputs_al = Xtr0_mat100.values
labels_al = Ytr0.values

inputs_tr = Xtr0_mat100.values[:1900]
labels_tr = Ytr0.values[:,1][:1900]

inputs_te = Xtr0_mat100.values[1900:]
labels_te = Ytr0.values[:,1][1900:]

In [ ]:
#Perform PCA with linear Kernel
pca_kernel = lker.LinearKernel()
pca_agent = pca.PCA()
pca_inputs_al = pca_agent.perform_PCA(inputs = inputs_al,
                                      kernel = pca_kernel,
                                      dim_num =3)

pca_inputs_tr = pca_inputs_al[:1900]
pca_inputs_te = pca_inputs_al[1900:]

In [ ]:
#Computing bandwidth for gaussian kernel as mean norms of all distances between vectors
n = pca_inputs_al.shape[0]
Inputs_dup = np.tile(pca_inputs_al[np.newaxis,:,:], reps = (n,1,1))
mean_distance = np.mean(np.linalg.norm(Inputs_dup - np.einsum('pnm-> npm', Inputs_dup), axis = 2))
del Inputs_dup

In [ ]:

lam=10e-4
gamma = 1
bandwidth = gamma*mean_distance
kernel_svm = gker.GaussianKernel(bandwidth)

In [ ]:
svmClassifier = svm.SVMClassifier(lam = lam, kernel = kernel_svm)
svmClassifier.fit(pca_inputs_tr, labels_tr.astype(np.double))

In [ ]:
accuracy = svmClassifier.compute_test_accuracy(pca_inputs_te, labels_te)

In [ ]:
print(accuracy)